In [1]:
import pandas as pd

building_and_light = pd.read_csv('building_and_light_data.csv')
weather = pd.read_csv('weather_data.csv')

building_and_light = building_and_light.drop(columns='Unnamed: 0')
weather = weather.drop(columns='Unnamed: 0')

In [2]:
merged_df = pd.merge(building_and_light, 
                     weather, 
                     on = ['Longitude', 'Latitude', 'datetime', 'UHI Index'])

In [6]:
merged_df.to_csv('light_building_weather_data.csv')

,Longitude,Latitude,datetime,UHI Index,building_area_25,building_area_50,building_area_100,B01,B02,B03,...,B08,B8A,B11,B12,Area,airtemp,relative humidity,avg wind speed,wind direction,solar flux
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,183.978594,1597.323120,11046.690248,841.5,1053.0,1155.0,...,1832.0,1709.0,1792.0,1495.5,Bronx,27.2,47.3,2.6,165,621
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,266.930647,1729.473614,10814.226850,841.5,1053.0,1155.0,...,1832.0,1709.0,1792.0,1495.5,Bronx,27.2,47.3,2.6,165,621
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,321.122238,1951.435804,10525.136565,841.5,646.0,823.0,...,2241.0,2251.0,1548.0,1135.0,Bronx,27.2,47.3,2.6,165,621
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,343.873068,2287.039351,10446.113598,841.5,625.0,766.0,...,2200.0,2251.0,1548.0,1135.0,Bronx,27.2,47.3,2.6,165,621
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,382.402733,2652.734741,10479.891453,841.5,659.5,763.0,...,2161.0,2186.0,1617.5,1207.5,Bronx,27.2,47.3,2.6,165,621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11224,-73.957050,40.790333,24-07-2021 15:57,0.972470,0.000000,0.000000,0.000000,462.5,491.0,725.5,...,3152.0,3446.5,1925.0,1072.5,Manhattan,26.8,46.7,3.4,196,605
11225,-73.957063,40.790308,24-07-2021 15:57,0.972470,0.000000,0.000000,0.000000,462.5,506.5,741.5,...,3572.0,3446.5,1925.0,1072.5,Manhattan,26.8,46.7,3.4,196,605
11226,-73.957093,40.790270,24-07-2021 15:57,0.981124,0.000000,0.000000,0.000000,462.5,506.5,741.5,...,3572.0,3446.5,1925.0,1072.5,Manhattan,26.8,46.7,3.4,196,605
11227,-73.957112,40.790253,24-07-2021 15:59,0.981245,0.000000,0.000000,0.000000,462.5,506.5,741.5,...,3572.0,3446.5,1925.0,1072.5,Manhattan,27.0,46.1,2.7,209,620


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
X = merged_df.drop(columns=['Longitude', 
                            'Latitude', 
                            'datetime', 
                            'UHI Index', 'Area'])
y = merged_df['UHI Index']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [23]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [300,400,500],
    'max_depth': [None, 30, 40, 50],
    'min_samples_leaf': [1],
    'max_features': ['sqrt']
}

# Initialize the model
rf = RandomForestRegressor()

# Setup GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='r2')

# Fit to training data
grid_search.fit(X_train, y_train)

# Output the best parameters
print("Best Hyperparameters:", grid_search.best_params_)




Best Hyperparameters: {'max_depth': 40, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'n_estimators': 300}


In [22]:
best_rf = grid_search.best_estimator_
outsample_predictions = best_rf.predict(X_test)
Y_test = y_test.tolist()
print("Test R^2 Scored: ",r2_score(Y_test, outsample_predictions))

Test R^2 Scored:  0.8118481459256778
